In [1]:
import numpy as np #Usado 
from scipy.optimize import curve_fit #Función para fijar una curva a la distribución de datos. Usado
import matplotlib.pyplot as plt #Usado
from scipy.interpolate import interp1d #Usado
from itertools import product #Usado
from scipy.integrate import quad #Usado
from scipy.integrate import dblquad
from scipy.fft import fht #Usado
import math #Usado
import time #Usaso para tomar el tiempo de procesamiento
data=np.loadtxt('explanatory10_pk.dat', unpack=True) #Importando los datos de P_peak obtenidos de CLASS del modelo
#lambda CDM en archivo .dat. Cambia respecto al modelo usado: LCDM, wCDM, CPL.
column=data
#Separando las dos listas anidadas de "data" y sujetandolas a un arreglo de dos listas separadas donde la primera son los datos
#de k y la segunda los valores respectivos del espectro de potencias P(k).
splits=np.array_split(column,2)
for array in splits:
    list(array)
x=column[0,] #Tomamos los datos de la primera lista anidada en "data" siendo los datos de k
y=column[1,] #Tomamos los datos de la segunda lista anidada en "data" siendo los datos de P(k)

In [2]:
#Tomando ahora el logaritmo los datos de "data" para hacer el ajuste polinomial
x_log=np.log(x)
y_log=np.log(y)

#Ajuste de un polinomio de 3er orden, definiendo la función "objective" a partir de la cual haremos el ajuste polinomial.
def straight(x_log, a2, b2):
    return (a2 * x_log) + b2

# curve fit
popt, _ = curve_fit(straight, x_log[:20], y_log[:20])
# summarize the parameter values
a2, b2 = popt

# curve fit
popt, _ = curve_fit(straight, x_log[-5:], y_log[-5:])
# summarize the parameter values
c2, d2 = popt

#Función de interpolación cúbica
f=interp1d(x,y,kind='cubic',bounds_error=False)
#Definiendo Plin a partir de la interpolación, con dos interpolaciones diferentes para cada extremo del P(k) obtenido
#de los datos de CLASS.
def Plin(k):
    if k<1 and k>x[0]:
        interpo=np.float(f(k))
    elif k<=x[0]:
        logk=np.log(k)
        logy=straight(logk, a2, b2)
        interpo=np.exp(logy)
    else:
        logk=np.log(k)
        logy=straight(logk, c2, d2)
        interpo=np.exp(logy)
    return interpo

Plin=np.vectorize(Plin)

In [3]:
#Transformación rápida de Hankel (fht) en Python para obtener los polos de la función de correlación
#Parámetros de la transformación rápida de Hankel
n=10000 
j=np.linspace(0, n, 10000) #Donde j va de 0 a n
#Para este caso seguiré tomando como k_max=1e+3, pero en principio debería de ser el valor máximo de los datos para k pbtenidos de CLASS
kmin=1e-6 #Es el valor mínimo de la lista de valores para k obtenidos de CLASS, con la suposición que los valores teóricos introducidos son en una lista del menor al mayor.
kmax=1e+4 #En principio kmax debería de ser kmax=x[-1]
kc=np.sqrt(kmin*kmax) #Obtenido de cálculos a mano al considerar k como mínima y máxima de 1e-5 y 1e+3. Dada esta k central es que sabemos de donde a donde corre el arreglo de datos de k, siendo en este caso n=10000 k's. 
jc=n/2 #Obtenido de cálculos
m=math.log10(kmax/kc) 
dln=(m/(n-jc))*np.log(10) #Obtenido de cálculos
kj=kc*np.exp((j-jc)*dln) #Dado por el método de transformación rápida de Hankel (fht) en Python
rc=1/kc #Dada la relación entre k y r
rj=rc*np.exp((j-jc)*dln)

#Para poder usar la función fth, necesitamos una lista (arreglo) de aj, por lo que primero proceso aj para los kj y así proceso
#la transformación rápida de Hankel. De lo contrario, si solo proceso un aj para un elemento de kj y eso lo introduzco en Aj, me marca error.
def aj(kj):
    return kj**(3/2)*Plin(kj)

ajlist=aj(kj) #Esto me arroja listas

In [6]:
#Obteniendo las funciones de correlación para el monopolo, siguiendo la fuente del paper de Kirkby, sin distorsión del redshift.
mu0=1/2
Aj0=fht(ajlist, dln, mu0) #Esta función de transformación arroja listas, por lo que, xi0 también será una lista.
xi0list=(1/(2*np.pi**2))*np.sqrt(np.pi/(2*rj**3))*Aj0

#Así, para poder obtener una función de xi0, hacemos una interpolación, a partir de la cual obtendremos la función.
g0=interp1d(rj,xi0list,kind='cubic',bounds_error=False)

#Esta es la función de correlación para l=0 del modelo fiducial usado (\LambdaCDM)
def xi0(rj):
    interp=np.float(g0(rj))
    return interp

xi0=np.vectorize(xi0)

#Datos de rj tomados para graficar la función de correlación (broad band) sin BAO y hacer el ajuste de sus parámetros.
rjcut=np.concatenate((rj[5698:5919], rj[6175:6279])) #Sin BAO

#Obtenido con 10000 datos de r en un intervalo de rj=[20,200] con un salto en el pico de BAO en los intervalos rj=50-83 y 150-190
xi0cut=np.concatenate((xi0list[5698:5919], xi0list[6175:6279]))

#Definiendo la función de correlación de ajuste para l=0.
def xi0fit(r, ci0, ci1, ci2, ci3, ci4):
    return ci0/(r**3) + ci1/(r**2) + ci2/r + ci3 + ci4*r

# curve fit
popt, _ = curve_fit(xi0fit, rjcut, xi0cut)
# summarize the parameter values
ci0, ci1, ci2, ci3, ci4 = popt

#Definiendo la función de correlación Smooth.
def xi0smooth(rj):
    if rj<=83 or rj>=150:
        xis=xi0(rj)
    else:
        xis=xi0fit(rj, ci0, ci1, ci2, ci3, ci4)
    return xis

xi0smooth=np.vectorize(xi0smooth)

In [7]:
#Tomando n, j y jc dados en la obtención de xi0
n=10000 
j=np.linspace(0, n, 10000) #Donde j va de 0 a n
rmin=1e-3
rmax=1e+5
rc_s=np.sqrt(rmin*rmax)
m_s=math.log10(rmax/rc_s)
dln_s=(m_s/(n-jc))*np.log(10)
rj2=rc_s*np.exp((j-jc)*dln_s)
kc_s=1/rc_s
kj2=kc_s*np.exp((j-jc)*dln_s)

def aj_s(rj2):
    return rj2**(3/2)*xi0smooth(rj2)

aj_s=np.vectorize(aj_s)

aj2list=aj_s(rj2) #Esto me arroja listas

mu0=1/2
Aj0_s=fht(aj2list, dln_s, mu0) #Esta función de transformación arroja listas, por lo que, xi0 también será una lista.
P0slist=np.sqrt((8*np.pi**3)/(kj2**3))*Aj0_s

In [8]:
np.savetxt('P0smooth_z234.txt', [kj2,P0slist])